### Evaluating the m5 limits ###

Create values to be used in `rubin_sim.utils.sysEngVals`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import syseng_throughputs as st
import os
from rubin_sim.phot_utils import Bandpass
from rubin_scheduler.data import get_data_dir

In [2]:
pd.set_option('display.precision', 3)

In [3]:
# Read bandpass information.
defaultDirs = st.setDefaultDirs()
addLosses = True
# Use the X=1.0 atmosphere!
atmosphere = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
hardware, system = st.buildHardwareAndSystem(defaultDirs, addLosses, atmosphereOverride=atmosphere)

In [4]:
# --note these are 1x30s and the overview paper uses 2x15s!!! 
# (this is used as the 'base exposure time' in sysEngVals and the m5 calculation from Cm/dCm_infinity)
# Note also this is using fiducial (old) values of the seeing to set the m5 values as well. 
# thus the resulting values are different than those reported in pstn-054, where the Cm/dCm reported values are 
# for the median delivered seeing in the opsim databases.

# sysengValues only records one exposure time (not per filter) so makes more sense to make everything 1x30
exptime = 30
nexp = 1
m5_std = st.makeM5(hardware, system, 
                   exptime={'u': exptime, 'g': exptime, 'r': exptime, 'i': exptime, 'z': exptime, 'y': exptime},
                   nexp={'u': nexp, 'g': nexp, 'r': nexp, 'i': nexp, 'z': nexp, 'y': nexp},
                   readnoise=8.8, othernoise=0, darkcurrent=0.2)

m5_std

,nexp,exptime,FWHMeff,FWHMgeom,skyMag,skyCounts,Zp_t,Tb,Sb,kAtm,gamma,Cm,dCm_infinity,dCm_double,m5,sourceCounts,m5_fid,m5_min
u,1.0,30.0,0.92,0.808,23.052,45.303,26.524,0.023,0.035,0.470,0.038,22.968,0.543,0.344,23.697,405.543,23.9,23.4
g,1.0,30.0,0.87,0.767,22.254,463.634,28.508,0.143,0.173,0.213,0.039,24.582,0.088,0.044,24.973,778.397,25.0,24.6
r,1.0,30.0,0.83,0.734,21.198,988.627,28.361,0.124,0.140,0.126,0.039,24.602,0.043,0.020,24.516,1035.302,24.7,24.3
i,1.0,30.0,0.80,0.710,20.463,1588.281,28.171,0.105,0.114,0.096,0.039,24.541,0.028,0.012,24.128,1243.517,24.0,23.6
z,1.0,30.0,0.78,0.693,19.606,2381.945,27.782,0.073,0.078,0.068,0.039,24.371,0.019,0.007,23.557,1470.048,23.3,22.9
y,1.0,30.0,0.76,0.677,18.602,2717.112,26.818,0.030,0.035,0.171,0.039,23.840,0.016,0.006,22.552,1526.132,22.1,21.7


In [5]:
fdir = os.path.join(get_data_dir(), "throughputs/baseline")
filter_list=["u", "g", "r", "i", "z", "y"]
eff_wavelens = {}
for f in filter_list:
    bp = Bandpass()
    bp.read_throughput(os.path.join(fdir, "total_" + f + ".dat"))
    eff_wavelens[f] = bp.calc_eff_wavelen()[1]


# Write out values of dictionaries that can be pasted into sims_utils
output_cols = ['Zp_t', 'Tb', 'gamma', 'kAtm', 'Cm', 'dCm_infinity', 'dCm_double', 'skyMag']
output_name = ['zp_t', 'tb', 'gamma', 'k_atm', 'cm', 'd_cm_infinity', 'd_cm_double', 'sky_mag']
for column, name in zip(output_cols, output_name):
    result = '{'
    for filtername, val in zip(m5_std.index, m5_std[column]):
        result += '"%s": %f, ' % (filtername, val)
    result += '}'
    result = result.replace(', }', '}')
    result = 'self.'+ name + ' = ' + result
    print(result)
print(f"self.exptime = {exptime}")

result = 'self.eff_wavelengths = {'
for filtername in eff_wavelens.keys():
    result += '"%s": %f, ' % (filtername, eff_wavelens[filtername])
result = result.replace(', }', '}')
result = result[:-2]+'}'
print(result)

self.zp_t = {"u": 26.524237, "g": 28.508375, "r": 28.360838, "i": 28.171396, "z": 27.782264, "y": 26.817819}
self.tb = {"u": 0.022928, "g": 0.142565, "r": 0.124451, "i": 0.104526, "z": 0.073041, "y": 0.030046}
self.gamma = {"u": 0.037534, "g": 0.038715, "r": 0.039034, "i": 0.039196, "z": 0.039320, "y": 0.039345}
self.k_atm = {"u": 0.470116, "g": 0.212949, "r": 0.126369, "i": 0.095764, "z": 0.068417, "y": 0.171009}
self.cm = {"u": 22.967681, "g": 24.582309, "r": 24.602134, "i": 24.541152, "z": 24.371077, "y": 23.840175}
self.d_cm_infinity = {"u": 0.543325, "g": 0.088310, "r": 0.043438, "i": 0.027510, "z": 0.018530, "y": 0.016283}
self.d_cm_double = {"u": 0.343781, "g": 0.043738, "r": 0.019756, "i": 0.011651, "z": 0.007261, "y": 0.006144}
self.sky_mag = {"u": 23.051983, "g": 22.253839, "r": 21.197579, "i": 20.462795, "z": 19.606305, "y": 18.601512}
self.exptime = 30
self.eff_wavelengths = {"u": 372.354597, "g": 480.687954, "r": 622.146782, "i": 755.898675, "z": 867.965175, "y": 975.34354